# Model Pruning Experiments

This notebook demonstrates pruning experiments on:
1. Vision Transformer (ViT)
2. Deep Compression AutoEncoder (DC-AE)

In [1]:
# Clone the repository
!git clone https://github.com/tahahah/model-pruning-experiments.git
%cd model-pruning-experiments

# Install dependencies
!pip install -r requirements.txt

Cloning into 'model-pruning-experiments'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 104 (delta 48), reused 73 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 457.37 KiB | 1.74 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/model-pruning-experiments
  Cloning https://github.com/mit-han-lab/efficientvit.git to /tmp/pip-req-build-u6i1_kl8
  Running command git clone --filter=blob:none --quiet https://github.com/mit-han-lab/efficientvit.git /tmp/pip-req-build-u6i1_kl8
  Resolved https://github.com/mit-han-lab/efficientvit.git to commit b94ff779828eea399c78f626b574da2d50ef2e49
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/alibaba/TinyNeuralNetwork.git to /tmp/pip-install-dkakei9i/tinyneuralnetwork_83827af5aecd4a8eb74a74e6225e538a
 

## 1. Vision Transformer (ViT) Pruning

In [ ]:
from vit_pruning_analysis import *

# Run ViT pruning analysis
main()

## 2. DC-AE Pruning

In [10]:
!git stash && git pull

Saved working directory and index state WIP on master: 74f3dfa fix: incorrect import utils
Already up to date.


In [ ]:
!python vae_pruning_analysis.py

Loading DC-AE model...

Analyzing original model...

Model Sparsity Analysis:
Total Parameters: 676,475,584
Non-zero Parameters: 676,475,584 (100.0%)
Zero Parameters: 0 (0.0%)

Top 10 Most Sparse Layers:
                                                                      Layer      Size   NonZero  Sparsity(%)
147                          decoder.project_out.op_list.2.conv.conv.weight     27648     27648          0.0
0                                       encoder.project_in.conv.conv.weight      1728      1728          0.0
1                         encoder.stages.1.op_list.0.main.conv1.conv.weight    589824    589824          0.0
2                         encoder.stages.1.op_list.0.main.conv2.conv.weight    589824    589824          0.0
132          decoder.stages.5.op_list.2.context_module.main.qkv.conv.weight   3145728   3145728          0.0
133         decoder.stages.5.op_list.2.context_module.main.proj.conv.weight   1048576   1048576          0.0
134  decoder.stages.5.op_list.2.l

## 3. Finetuning DC AE

In [2]:
!pip install datasets
!wandb login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: h

In [8]:
!git reset --hard && git pull


HEAD is now at f7af0b2 feat: setup code
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 9 (delta 6), reused 7 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (9/9), 1.36 KiB | 464.00 KiB/s, done.
From https://github.com/tahahah/model-pruning-experiments
   f7af0b2..c4924ae  master     -> origin/master
Updating f7af0b2..c4924ae
Fast-forward
 dcaecore/setup.py       | 56 +++++++++++++++++---------------------------------------
 dcaecore/train_dc_ae.py | 63 ++++++++++++++++++++++++---------------------------------------
 2 files changed, 41 insertions(+), 78 deletions(-)


In [10]:
import os
os.environ["RANK"]="1"
os.environ["WORLD_SIZE"]="1"
!python dcaecore/train_dc_ae.py \
    --config dcaecore/config.yaml \
    --pretrained mit-han-lab/dc-ae-f32c32-in-1.0 \
    --output_dir ./outputs \
    --seed 42 \
    --gpu 0  # or omit this to use the default GPU

Traceback (most recent call last):
  File "/content/model-pruning-experiments/dcaecore/train_dc_ae.py", line 191, in <module>
    main()
  File "/content/model-pruning-experiments/dcaecore/train_dc_ae.py", line 136, in main
    logger = setup_logger(args.output_dir)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/model-pruning-experiments/dcaecore/train_dc_ae.py", line 52, in setup_logger
    file_handler = logging.FileHandler(log_file)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/logging/__init__.py", line 1181, in __init__
    StreamHandler.__init__(self, self._open())
                                 ^^^^^^^^^^^^
  File "/usr/lib/python3.11/logging/__init__.py", line 1213, in _open
    return open_func(self.baseFilename, self.mode,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/model-pruning-experiments/outputs/training_20250213_071837.log'


## View Results

The results are saved in the `output` directory:
- Pruning analysis reports
- Reconstructed images
- Model statistics